In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'legend.fontsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

print("Libraries loaded successfully!")

## 1. Load Simulation Data

In [ ]:
# Load all CSV files
scalability_df = pd.read_csv('scalability_analysis.csv')
aggregated_df = pd.read_csv('aggregated_latency_results.csv')
comparison_df = pd.read_csv('ds_vs_sps_comparison.csv')

print("=" * 60)
print("SCALABILITY ANALYSIS DATA")
print("=" * 60)
print(f"Shape: {scalability_df.shape}")
print(f"\nColumns: {list(scalability_df.columns)}")
print(f"\nData:")
display(scalability_df)

In [ ]:
print("=" * 60)
print("AGGREGATED LATENCY RESULTS")
print("=" * 60)
print(f"Shape: {aggregated_df.shape}")
print(f"\nColumns: {list(aggregated_df.columns)}")
display(aggregated_df.head(20))

In [ ]:
print("=" * 60)
print("DS vs SPS COMPARISON (Theoretical vs Simulated)")
print("=" * 60)
display(comparison_df)

## 2. Main Visualization: DS vs SPS Latency (Figure 2 Recreation)

This plot recreates Figure 2 from the IEEE IDAACS 2018 paper, showing the uplink latency comparison between Dynamic Scheduling and Semi-Persistent Scheduling as the number of UEs increases.

In [ ]:
# Create the main comparison plot (Figure 2 from paper)
fig, ax = plt.subplots(figsize=(12, 7))

# Plot DS and SPS latency curves
ax.plot(scalability_df['num_ues'], scalability_df['ds_mean_latency_ms'], 
        'b-o', linewidth=2.5, markersize=8, label='Dynamic Scheduling (DS)', 
        markerfacecolor='blue', markeredgecolor='darkblue', markeredgewidth=1.5)

ax.plot(scalability_df['num_ues'], scalability_df['sps_mean_latency_ms'], 
        'r-s', linewidth=2.5, markersize=8, label='Semi-Persistent Scheduling (SPS)',
        markerfacecolor='red', markeredgecolor='darkred', markeredgewidth=1.5)

# Add horizontal threshold lines
ax.axhline(y=10, color='green', linestyle='--', linewidth=2, 
           label='10 ms Strict Latency Requirement', alpha=0.8)
ax.axhline(y=50, color='orange', linestyle='--', linewidth=2, 
           label='50 ms Relaxed Latency Requirement', alpha=0.8)

# Formatting
ax.set_xlabel('Number of UEs', fontsize=14, fontweight='bold')
ax.set_ylabel('Average Uplink Latency (ms)', fontsize=14, fontweight='bold')
ax.set_title('NB-IoT Uplink Latency: Dynamic Scheduling vs Semi-Persistent Scheduling\n(IEEE IDAACS 2018 Paper Reproduction)', 
             fontsize=16, fontweight='bold', pad=15)

# Set axis limits and ticks
ax.set_xlim(0, 105)
ax.set_ylim(0, 200)
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.yaxis.set_major_locator(MultipleLocator(20))

# Grid and legend
ax.grid(True, alpha=0.3, linestyle='-')
ax.legend(loc='upper left', framealpha=0.95, edgecolor='black')

# Add annotation for key insight
ax.annotate('SPS reduces latency\nby ~47% on average', 
            xy=(50, 40), fontsize=11, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('figure2_ds_vs_sps_latency.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Figure saved as 'figure2_ds_vs_sps_latency.png'")

## 3. Latency Improvement Analysis

In [ ]:
# Calculate improvement metrics
scalability_df['improvement_factor'] = scalability_df['ds_mean_latency_ms'] / scalability_df['sps_mean_latency_ms']
scalability_df['improvement_percentage'] = ((scalability_df['ds_mean_latency_ms'] - scalability_df['sps_mean_latency_ms']) 
                                            / scalability_df['ds_mean_latency_ms'] * 100)

# Display improvement analysis
print("=" * 70)
print("LATENCY IMPROVEMENT ANALYSIS: SPS vs DS")
print("=" * 70)
improvement_table = scalability_df[['num_ues', 'ds_mean_latency_ms', 'sps_mean_latency_ms', 
                                     'latency_difference_ms', 'improvement_factor', 'improvement_percentage']].copy()
improvement_table.columns = ['UEs', 'DS Latency (ms)', 'SPS Latency (ms)', 
                              'Reduction (ms)', 'Improvement Factor', 'Improvement (%)']
display(improvement_table.round(2))

In [ ]:
# Bar chart: Latency Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Grouped bar chart
x = np.arange(len(scalability_df))
width = 0.35

bars1 = axes[0].bar(x - width/2, scalability_df['ds_mean_latency_ms'], width, 
                    label='Dynamic Scheduling', color='steelblue', edgecolor='black')
bars2 = axes[0].bar(x + width/2, scalability_df['sps_mean_latency_ms'], width, 
                    label='SPS', color='coral', edgecolor='black')

axes[0].set_xlabel('Number of UEs', fontweight='bold')
axes[0].set_ylabel('Latency (ms)', fontweight='bold')
axes[0].set_title('Latency Comparison: DS vs SPS', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(scalability_df['num_ues'])
axes[0].legend()
axes[0].axhline(y=10, color='green', linestyle='--', alpha=0.7, label='10ms threshold')
axes[0].axhline(y=50, color='orange', linestyle='--', alpha=0.7, label='50ms threshold')

# Right: Improvement percentage
colors = ['green' if x > 0 else 'red' for x in scalability_df['improvement_percentage']]
axes[1].bar(scalability_df['num_ues'], scalability_df['improvement_percentage'], 
            color=colors, edgecolor='black', alpha=0.8)
axes[1].set_xlabel('Number of UEs', fontweight='bold')
axes[1].set_ylabel('Improvement (%)', fontweight='bold')
axes[1].set_title('SPS Latency Improvement over DS (%)', fontweight='bold')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('latency_comparison_bars.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Latency Requirement Compliance Analysis

In [ ]:
# Determine max UEs meeting requirements
print("=" * 70)
print("LATENCY REQUIREMENT COMPLIANCE ANALYSIS")
print("=" * 70)

# 10ms requirement
ds_10ms = scalability_df[scalability_df['ds_mean_latency_ms'] <= 10]['num_ues'].max()
sps_10ms = scalability_df[scalability_df['sps_mean_latency_ms'] <= 10]['num_ues'].max()

# 50ms requirement  
ds_50ms = scalability_df[scalability_df['ds_mean_latency_ms'] <= 50]['num_ues'].max()
sps_50ms = scalability_df[scalability_df['sps_mean_latency_ms'] <= 50]['num_ues'].max()

print(f"\n📊 Maximum UEs Meeting Latency Requirements:")
print(f"{'─' * 50}")
print(f"{'Requirement':<25} {'DS':<15} {'SPS':<15}")
print(f"{'─' * 50}")
print(f"{'10 ms (Strict)':<25} {ds_10ms if pd.notna(ds_10ms) else 0:<15} {sps_10ms if pd.notna(sps_10ms) else 0:<15}")
print(f"{'50 ms (Relaxed)':<25} {ds_50ms if pd.notna(ds_50ms) else 0:<15} {sps_50ms if pd.notna(sps_50ms) else 0:<15}")
print(f"{'─' * 50}")

# Calculate capacity improvement
if pd.notna(sps_10ms) and pd.notna(ds_10ms) and ds_10ms > 0:
    capacity_10ms = ((sps_10ms - ds_10ms) / ds_10ms) * 100
    print(f"\n🚀 SPS provides {capacity_10ms:.1f}% more capacity under 10ms requirement")

if pd.notna(sps_50ms) and pd.notna(ds_50ms) and ds_50ms > 0:
    capacity_50ms = ((sps_50ms - ds_50ms) / ds_50ms) * 100  
    print(f"🚀 SPS provides {capacity_50ms:.1f}% more capacity under 50ms requirement")

In [ ]:
# Visualize compliance
fig, ax = plt.subplots(figsize=(12, 6))

# Create stacked area for regions
ax.fill_between(scalability_df['num_ues'], 0, 10, alpha=0.2, color='green', label='Strict (≤10ms)')
ax.fill_between(scalability_df['num_ues'], 10, 50, alpha=0.2, color='yellow', label='Relaxed (10-50ms)')
ax.fill_between(scalability_df['num_ues'], 50, 200, alpha=0.2, color='red', label='Non-compliant (>50ms)')

# Plot latency curves
ax.plot(scalability_df['num_ues'], scalability_df['ds_mean_latency_ms'], 
        'b-o', linewidth=2.5, markersize=7, label='DS Latency')
ax.plot(scalability_df['num_ues'], scalability_df['sps_mean_latency_ms'], 
        'r-s', linewidth=2.5, markersize=7, label='SPS Latency')

ax.set_xlabel('Number of UEs', fontsize=13, fontweight='bold')
ax.set_ylabel('Average Latency (ms)', fontsize=13, fontweight='bold')
ax.set_title('Latency Requirement Compliance Regions', fontsize=14, fontweight='bold')
ax.set_xlim(0, 105)
ax.set_ylim(0, 200)
ax.legend(loc='upper left', ncol=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('compliance_regions.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Statistical Distribution Analysis

In [ ]:
# Prepare data from aggregated results
scalability_data = aggregated_df[aggregated_df['scenario'] == 'scalability'].copy()

# Separate DS and SPS data
ds_data = scalability_data[scalability_data['scheduling_type'] == 'DS']
sps_data = scalability_data[scalability_data['scheduling_type'] == 'SPS']

print("=" * 70)
print("STATISTICAL SUMMARY")
print("=" * 70)

print("\n📈 Dynamic Scheduling Statistics:")
display(ds_data[['num_ues', 'mean_latency_ms', 'std_dev_latency_ms', 'min_latency_ms', 'max_latency_ms']].round(2))

print("\n📈 Semi-Persistent Scheduling Statistics:")
display(sps_data[['num_ues', 'mean_latency_ms', 'std_dev_latency_ms', 'min_latency_ms', 'max_latency_ms']].round(2))

In [ ]:
# Create error bar plot with std deviation
fig, ax = plt.subplots(figsize=(12, 7))

# Plot with error bars
ax.errorbar(ds_data['num_ues'], ds_data['mean_latency_ms'], 
            yerr=ds_data['std_dev_latency_ms'], 
            fmt='b-o', linewidth=2, markersize=7, capsize=4, capthick=2,
            label='DS (mean ± std)', ecolor='lightblue')

ax.errorbar(sps_data['num_ues'], sps_data['mean_latency_ms'], 
            yerr=sps_data['std_dev_latency_ms'], 
            fmt='r-s', linewidth=2, markersize=7, capsize=4, capthick=2,
            label='SPS (mean ± std)', ecolor='lightcoral')

ax.axhline(y=10, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axhline(y=50, color='orange', linestyle='--', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Number of UEs', fontsize=13, fontweight='bold')
ax.set_ylabel('Latency (ms)', fontsize=13, fontweight='bold')
ax.set_title('Latency Distribution with Standard Deviation', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 105)
ax.set_ylim(0, 200)

plt.tight_layout()
plt.savefig('latency_with_std.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Improvement Factor Visualization

In [ ]:
# Create improvement factor visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Improvement factor line plot
axes[0].plot(scalability_df['num_ues'], scalability_df['improvement_factor'], 
             'g-^', linewidth=2.5, markersize=9, markerfacecolor='lime', 
             markeredgecolor='darkgreen', markeredgewidth=1.5)
axes[0].axhline(y=1.0, color='red', linestyle='--', linewidth=1.5, 
                label='Break-even (factor=1.0)')
axes[0].fill_between(scalability_df['num_ues'], 1.0, scalability_df['improvement_factor'],
                     where=scalability_df['improvement_factor'] > 1.0,
                     alpha=0.3, color='green', label='SPS Better')
axes[0].fill_between(scalability_df['num_ues'], 1.0, scalability_df['improvement_factor'],
                     where=scalability_df['improvement_factor'] < 1.0,
                     alpha=0.3, color='red', label='DS Better')
axes[0].set_xlabel('Number of UEs', fontweight='bold')
axes[0].set_ylabel('Improvement Factor (DS/SPS)', fontweight='bold')
axes[0].set_title('SPS Improvement Factor vs Number of UEs', fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 105)

# Right: Latency reduction in ms
colors = ['green' if x > 0 else 'red' for x in scalability_df['latency_difference_ms']]
axes[1].bar(scalability_df['num_ues'], scalability_df['latency_difference_ms'], 
            color=colors, edgecolor='black', alpha=0.8, width=5)
axes[1].set_xlabel('Number of UEs', fontweight='bold')
axes[1].set_ylabel('Latency Reduction (ms)', fontweight='bold')
axes[1].set_title('Absolute Latency Reduction (DS - SPS)', fontweight='bold')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('improvement_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Theoretical vs Simulated Comparison

In [ ]:
# Display theoretical vs simulated comparison
print("=" * 70)
print("THEORETICAL vs SIMULATED LATENCY COMPARISON")
print("=" * 70)
print("\n📚 From IEEE IDAACS 2018 Paper (Equation 2 & 3):")
print("   DS Latency  = (SR_period/2) + SG_delay + UL_TX + NbRep × TTI × (n-1)/n_RU")
print("   SPS Latency = (SPS_period/2) + UL_TX + NbRep × TTI × (n-1)/n_RU")
print("\n   Where: TTI=1ms, SR_period=5ms, SG_delay=4ms, SPS_period=10ms")

display(comparison_df)

In [ ]:
# Create theoretical vs simulated bar chart
if len(comparison_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(2)
    width = 0.25
    
    theoretical = [comparison_df['ds_theoretical_ms'].values[0], 
                   comparison_df['sps_theoretical_ms'].values[0]]
    simulated = [comparison_df['ds_simulated_ms'].values[0], 
                 comparison_df['sps_simulated_ms'].values[0]]
    
    bars1 = ax.bar(x - width/2, theoretical, width, label='Theoretical', 
                   color='steelblue', edgecolor='black')
    bars2 = ax.bar(x + width/2, simulated, width, label='Simulated', 
                   color='coral', edgecolor='black')
    
    ax.set_ylabel('Latency (ms)', fontweight='bold')
    ax.set_title('Single UE Latency: Theoretical vs Simulated', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(['Dynamic Scheduling', 'Semi-Persistent Scheduling'])
    ax.legend()
    
    # Add value labels
    for bar in bars1:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords='offset points', ha='center', fontsize=11)
    for bar in bars2:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords='offset points', ha='center', fontsize=11)
    
    ax.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('theoretical_vs_simulated.png', dpi=300, bbox_inches='tight')
    plt.show()

## 8. Summary Dashboard

In [ ]:
# Create comprehensive dashboard
fig = plt.figure(figsize=(16, 12))

# Main plot: DS vs SPS latency
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(scalability_df['num_ues'], scalability_df['ds_mean_latency_ms'], 
         'b-o', linewidth=2, markersize=6, label='DS')
ax1.plot(scalability_df['num_ues'], scalability_df['sps_mean_latency_ms'], 
         'r-s', linewidth=2, markersize=6, label='SPS')
ax1.axhline(y=10, color='green', linestyle='--', alpha=0.7)
ax1.axhline(y=50, color='orange', linestyle='--', alpha=0.7)
ax1.set_xlabel('Number of UEs')
ax1.set_ylabel('Latency (ms)')
ax1.set_title('DS vs SPS Latency Comparison', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Improvement percentage
ax2 = fig.add_subplot(2, 2, 2)
colors = ['forestgreen' if x > 0 else 'crimson' for x in scalability_df['improvement_percentage']]
ax2.bar(scalability_df['num_ues'], scalability_df['improvement_percentage'], 
        color=colors, edgecolor='black', width=5)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Number of UEs')
ax2.set_ylabel('Improvement (%)')
ax2.set_title('SPS Latency Improvement (%)', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Improvement factor
ax3 = fig.add_subplot(2, 2, 3)
ax3.plot(scalability_df['num_ues'], scalability_df['improvement_factor'], 
         'g-^', linewidth=2, markersize=7)
ax3.axhline(y=1.0, color='red', linestyle='--', linewidth=1.5)
ax3.set_xlabel('Number of UEs')
ax3.set_ylabel('Improvement Factor (DS/SPS)')
ax3.set_title('SPS Improvement Factor', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Summary statistics text
ax4 = fig.add_subplot(2, 2, 4)
ax4.axis('off')

# Calculate summary stats
avg_improvement = scalability_df['improvement_percentage'].mean()
max_improvement = scalability_df['improvement_percentage'].max()
avg_factor = scalability_df[scalability_df['improvement_factor'] > 1]['improvement_factor'].mean()

summary_text = f"""
╔══════════════════════════════════════════════════════════╗
║           NB-IoT SIMULATION SUMMARY REPORT               ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  📊 KEY FINDINGS:                                        ║
║  ─────────────────────────────────────────────────────   ║
║  • Average Latency Improvement: {avg_improvement:>6.1f}%                  ║
║  • Maximum Latency Improvement: {max_improvement:>6.1f}%                  ║
║  • Average Improvement Factor:  {avg_factor:>6.2f}x                   ║
║                                                          ║
║  📈 CAPACITY ANALYSIS:                                   ║
║  ─────────────────────────────────────────────────────   ║
║  • Max UEs at ≤10ms (DS):  {ds_10ms if pd.notna(ds_10ms) else 0:>4} UEs                       ║
║  • Max UEs at ≤10ms (SPS): {sps_10ms if pd.notna(sps_10ms) else 0:>4} UEs                       ║
║  • Max UEs at ≤50ms (DS):  {ds_50ms if pd.notna(ds_50ms) else 0:>4} UEs                       ║
║  • Max UEs at ≤50ms (SPS): {sps_50ms if pd.notna(sps_50ms) else 0:>4} UEs                       ║
║                                                          ║
║  ✅ CONCLUSION:                                          ║
║  SPS significantly reduces latency, especially as        ║
║  the number of UEs increases, making it ideal for        ║
║  massive IoT deployments with latency constraints.       ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
"""

ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes, fontsize=10,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.suptitle('NB-IoT SPS vs DS Scheduling - Complete Analysis Dashboard', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('complete_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("✅ All visualizations saved successfully!")
print("="*60)
print("\nGenerated files:")
print("  • figure2_ds_vs_sps_latency.png")
print("  • latency_comparison_bars.png")
print("  • compliance_regions.png")
print("  • latency_with_std.png")
print("  • improvement_analysis.png")
print("  • theoretical_vs_simulated.png")
print("  • complete_dashboard.png")

## 9. Key Observations & Conclusions

### Findings from the Simulation:

1. **SPS Outperforms DS at Scale**: As the number of UEs increases, SPS maintains significantly lower latency compared to DS due to elimination of scheduling signaling overhead.

2. **Latency Requirements**: 
   - For strict 10ms requirements, SPS supports more UEs than DS
   - For relaxed 50ms requirements, SPS provides substantial additional capacity

3. **Scalability Advantage**: The improvement factor increases with more UEs, demonstrating SPS's superior scalability for massive IoT deployments.

4. **Trade-offs**: At very low UE counts (1 UE), DS may perform comparably or slightly better due to the simplified resource allocation model.

### Alignment with IEEE IDAACS 2018 Paper:
- The simulation confirms the paper's theoretical predictions about SPS benefits
- The latency curves follow the expected trends from Equations 2 and 3
- SPS eliminates the ~7ms signaling overhead present in DS